<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesion2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilovegabarbosa/

---

## Guía: Ejecutar Modelos de Lenguaje con Llama-2-7b

Este notebook presenta **4 métodos diferentes** para ejecutar modelos de lenguaje:

1. **Transformers Pipeline** - Implementación básica
2. **Accelerate + BitsAndBytes** - Optimizado para GPU
3. **Ollama** - Ejecución local simplificada  
4. **Groq** - API en la nube

### Requisitos
- **GPU:** Tesla T4 mínimo (Colab gratuito)
- **APIs:** Hugging Face token, Groq API key (opcional)

## Personalización

```python
# Modificar parámetros en cualquier método
MAX_TOKENS = 1024      # Respuestas más largas
TEMPERATURE = 0.9      # Más creatividad
SYSTEM_PROMPT = "Tu prompt personalizado"
```

### Configuración APIs
- **Hugging Face:**
  1. [Crear token](https://huggingface.co/settings/tokens)
  2. En Colab: 🔑 Secrets → Agregar `HF_TOKEN` → Pegar tu token
- **Groq:**
  1. [Registrarse](https://console.groq.com/)
  2. En Colab: 🔑 Secrets → Agregar `GROQ_KEY` → Pegar tu API key

Cada método es **independiente** y puede ejecutarse por separado.

In [ ]:
# MÉTODO 1: TRANSFORMERS PIPELINE
# Implementación básica sin optimizaciones

# 1. Instalación de dependencias
!pip install transformers torch -q

# 2. Importar bibliotecas
from transformers import pipeline
import torch
from google.colab import userdata

# 3. Obtener token de Hugging Face desde secretos de Colab
HF_TOKEN = userdata.get('HF_TOKEN')

# 4. Parámetros configurables
MAX_TOKENS = 512
TEMPERATURE = 0.7
SYSTEM_PROMPT = "Eres un asistente útil y educativo. Responde de manera clara y concisa."

# 5. Cargar modelo usando pipeline con token de secretos
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN  # Token desde secretos de Colab
)

# 5. Función para generar respuestas
def generar_respuesta(user_input, system_prompt=SYSTEM_PROMPT, max_tokens=MAX_TOKENS, temperature=TEMPERATURE):
    """
    Genera respuesta usando Transformers Pipeline
    """
    # Crear estructura de mensajes
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    # Generar respuesta
    result = pipe(
        messages,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True
    )

    # Extraer texto de respuesta
    response = result[0]['generated_text'][-1]['content']
    return response

# 6. Ejemplo de uso
user_question = "¿Qué es el machine learning?"
respuesta = generar_respuesta(user_question)
print(f"Pregunta: {user_question}")
print(f"Respuesta: {respuesta}")

In [ ]:
# MÉTODO 2: ACCELERATE + BITSANDBYTES
# Optimizado para GPU con cuantización

# 1. Instalación de dependencias
!pip install transformers torch accelerate bitsandbytes -q

# 2. Importar bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from google.colab import userdata

# 3. Obtener token de Hugging Face desde secretos de Colab
HF_TOKEN = userdata.get('HF_TOKEN')

# 4. Parámetros configurables
MAX_TOKENS = 512
TEMPERATURE = 0.7
SYSTEM_PROMPT = "Eres un asistente útil y educativo. Responde de manera clara y concisa."

# 4. Configurar cuantización 8-bit para optimizar memoria
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

# 6. Cargar tokenizador con token de secretos
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    token=HF_TOKEN
)
tokenizer.pad_token = tokenizer.eos_token

# 7. Cargar modelo con optimizaciones y token de secretos
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    token=HF_TOKEN  # Token desde secretos de Colab
)

# 7. Función para generar respuestas
def generar_respuesta_optimizada(user_input, system_prompt=SYSTEM_PROMPT, max_tokens=MAX_TOKENS, temperature=TEMPERATURE):
    """
    Genera respuesta usando modelo optimizado
    """
    # Formato específico de Llama-2 para chat
    formatted_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_input} [/INST]"

    # Tokenizar entrada
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    # Generar respuesta con optimizaciones
    with torch.inference_mode():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decodificar solo la nueva parte generada
    response = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )

    return response

# 8. Ejemplo de uso
user_question = "Explica las diferencias entre deep learning y machine learning tradicional."
respuesta = generar_respuesta_optimizada(user_question)
print(f"Pregunta: {user_question}")
print(f"Respuesta: {respuesta}")

In [ ]:
# MÉTODO 3: OLLAMA
# Ejecución local simplificada

# 1. Importar bibliotecas
import subprocess
import json
import time

# 2. Parámetros configurables
MAX_TOKENS = 512
TEMPERATURE = 0.7
SYSTEM_PROMPT = "Eres un asistente útil y educativo. Responde de manera clara y concisa."
MODELO_OLLAMA = "llama2:7b"

# 3. Instalar Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 4. Configurar servidor Ollama
!pkill ollama || true  # Detener procesos existentes
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 &  # Iniciar servidor

# 5. Esperar que el servidor inicie
time.sleep(20)  # Aumentar tiempo de espera

# 6. Verificar servidor
!curl -s http://localhost:11434/api/tags

# 7. Descargar modelo Llama-2
!/usr/local/bin/ollama pull {MODELO_OLLAMA}  # Usar ruta completa

# 8. Función para generar respuestas
def generar_respuesta_ollama(user_input, system_prompt=SYSTEM_PROMPT, max_tokens=MAX_TOKENS, temperature=TEMPERATURE):
    """
    Genera respuesta usando Ollama
    """
    # Combinar system prompt con entrada del usuario
    full_prompt = f"{system_prompt}\\n\\nUsuario: {user_input}\\nAsistente:"

    # Crear comando curl para API de Ollama
    comando = f'''curl -s http://localhost:11434/api/generate -d '{{"model": "{MODELO_OLLAMA}", "prompt": "{full_prompt}", "stream": false, "options": {{"temperature": {temperature}, "num_predict": {max_tokens}}}}}'
    '''

    try:
        # Ejecutar comando y obtener respuesta
        resultado = subprocess.check_output(comando, shell=True, text=True)
        respuesta_json = json.loads(resultado)
        response = respuesta_json.get('response', 'No se obtuvo respuesta')
        return response

    except Exception as e:
        return f"Error: {e}"

# 9. Ejemplo de uso
user_question = "¿Cuáles son las ventajas de usar redes neuronales convolucionales?"
respuesta = generar_respuesta_ollama(user_question)
print(f"Pregunta: {user_question}")
print(f"Respuesta: {respuesta}")

In [ ]:
# MÉTODO 4: GROQ API
# API en la nube de alta velocidad

# 1. Instalación de dependencias
!pip install groq -q

# 2. Importar bibliotecas
from groq import Groq
from google.colab import userdata

# 3. Obtener API Key de Groq desde secretos de Colab
GROQ_API_KEY = userdata.get('GROQ_KEY')

# 4. Parámetros configurables
MAX_TOKENS = 512
TEMPERATURE = 0.7
SYSTEM_PROMPT = "Eres un asistente útil y educativo. Responde de manera clara y concisa."
MODELO_GROQ = "llama3-8b-8192"  # Modelo actualizado disponible

# 5. Inicializar cliente Groq
if GROQ_API_KEY:
    client = Groq(api_key=GROQ_API_KEY)
    print("Cliente Groq inicializado")
else:
    print("Error: GROQ_KEY no encontrado en secretos de Colab")

# 6. Función para generar respuestas
def generar_respuesta_groq(user_input, system_prompt=SYSTEM_PROMPT, max_tokens=MAX_TOKENS, temperature=TEMPERATURE):
    """
    Genera respuesta usando Groq API
    """
    try:
        completion = client.chat.completions.create(
            model=MODELO_GROQ,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return completion.choices[0].message.content

    except Exception as e:
        return f"Error: {e}"

# 7. Ejemplo de uso
user_question = "¿Qué son los transformers en procesamiento de lenguaje natural?"
respuesta = generar_respuesta_groq(user_question)
print(f"Pregunta: {user_question}")
print(f"Respuesta: {respuesta}")

# Comparación de Métodos

| Método | Velocidad | Uso GPU | Configuración | Costo | Ideal para |
|--------|-----------|---------|---------------|-------|------------|
| Pipeline | Media | Alta | Fácil | Gratis | Prototipos |
| Optimizado | Alta | Media | Intermedia | Gratis | Producción local |
| Ollama | Media | Baja | Fácil | Gratis | Desarrollo |
| Groq | Muy Alta | Ninguna | Fácil | Freemium | Apps rápidas |

## Recomendaciones

- **Para aprender:** Pipeline
- **Para máximo rendimiento:** Optimizado  
- **Para desarrollo local:** Ollama
- **Para producción:** Groq

